In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter

In [ ]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

In [ ]:
data.describe()

# Предобработка

In [ ]:
# создадим словарь для ответов
answers = {}

# создадим колонку, которая будет считать прибыль фильмов:
data["profit"] = data['revenue'] - data['budget']

# в данном датасете, изначально, объекты в колонке 'release_date' относились к типу str,
# поэтому преобразуем их в удобный для нас формат datetime:
data['release_date'] = pd.to_datetime(data['release_date'])

# создадим колонку, которая посчитает количество символов в назавнии фильмов:
data['lenght_title'] = data.original_title.map(lambda x: len(x))

# создадим колонку, которая посчитает количество слов в описании фильмов:
data['lenght_overview'] = data.overview.map(lambda x: len(x.split(' ')))

# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [ ]:

answers['1'] = '5. Pirates of the Caribbean: On Stranger Tides (tt1298650)'
# "+"

In [ ]:
data[data.budget == data.budget.max()]

# 2. Какой из фильмов самый длительный (в минутах)?

In [ ]:
answers['2'] = '2. Gods and Generals (tt0279111)'
# "+"

In [ ]:
data[data.runtime == data.runtime.max()]

# 3. Какой из фильмов самый короткий (в минутах)?

In [ ]:
answers['3'] = '3. Winnie the Pooh (tt1449283)'
# "+"

In [ ]:
data[data.runtime == data.runtime.min()]

# 4. Какова средняя длительность фильмов?

In [ ]:
answers['4'] = '2. 110'
# "+"

In [ ]:
data.runtime.mean()

# 5. Каково медианное значение длительности фильмов? 

In [ ]:
answers['5'] = '1. 107'
# "+"

In [ ]:
data.runtime.median()

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [ ]:
answers['6'] = '5. Avatar (tt0499549)'
# "+"

In [ ]:
data[data.profit == data.profit.max()]

# 7. Какой фильм самый убыточный? 

In [ ]:
answers['7'] = '5. The Lone Ranger (tt1210819)'
# "+"

In [ ]:
data[data.profit == data.profit.min()]

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [ ]:
answers['8'] = '1. 1478'
# "+"

In [ ]:
len(data[data['profit'] > 0])

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [ ]:
answers['9'] = '4. The Dark Knight (tt0468569)'
# "+"

In [ ]:
data.loc[data.release_year == 2008].sort_values(
    by='profit', ascending=False).head(1)

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [ ]:
answers['10'] = '5. The Lone Ranger (tt1210819)'
# "+"

In [ ]:
data[(data.release_year > 2012) &
     (data.release_year <= 2014)][['imdb_id', 'revenue',
                                   'profit', 'original_title', ]].sort_values('profit', ascending=True). head(1)

# 11. Какого жанра фильмов больше всего?

In [ ]:
answers['11'] = '3. Drama'
# "+"

In [ ]:
geners = pd.Series([x.split('|')
                    for x in data['genres']]).explode().value_counts()
geners

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [ ]:
answers['12'] = '1. Drama'
# "+"

In [ ]:
positive_profit = data[data.profit > 0]
genres_profit = positive_profit.genres.str.split(
    '|', expand=True).stack().value_counts()
genres_profit

# 13. У какого режиссера самые большие суммарные кассовые сбооры?

In [ ]:
answers['13'] = '5. Peter Jackson'
# "+"

In [ ]:
def director_revenue(df):
    '''Создаём функцию, которая суммирует прибыльность каждого режиссёра отдельно'''
    summ = Counter()
    for i in range(len(df)):
        for j in df.iloc[i].director.split('|'):
            summ[j] += df.iloc[i].revenue
    return pd.DataFrame.from_dict(summ, orient='index', columns=['Summ'])


display(director_revenue(data[data.revenue > 0]
                         ).sort_values(by=['Summ'], ascending=False))

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [ ]:
answers['14'] = '3. Robert Rodriguez' 
# "+"

In [ ]:
grouped1 = data[data.genres.str.contains("Action", na=False)]
grouped2 = grouped1['director'].str.split(
    '|', expand=True).stack().value_counts()
grouped2

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [ ]:
answers['15'] = '3. Chris Hemsworth'
# "+"

In [ ]:
data_1 = data[data.release_year == 2012][['cast', 'revenue']]
data_1.cast = data_1.cast.apply(lambda s: s.split('|'))
data_sub_exp = data_1.explode('cast')
data_sub_exp.groupby(by='cast').revenue.sum().sort_values(ascending=False)

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [ ]:
answers['16'] = '3. Matt Damon'
# "+"

In [ ]:
def counter(movie_bd, x):
    '''Создаём функцию, которая вернёт нам количество фильмов с заданными параметрами'''
    data_plot = movie_bd[x].str.cat(sep='|')
    dat = pd.Series(data_plot.split('|'))
    info = dat.value_counts(ascending=False)
    return info


sum_gen = counter(data[data['budget'] > data['budget'].mean()], 'cast')
sum_gen

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [ ]:
answers['17'] = '2. Action' 
# "+"

In [ ]:
grouped1 = data[data.cast.str.contains("Nicolas Cage", na=False)]
grouped2 = grouped1['genres'].str.split(
    '|', expand=True).stack().value_counts()
grouped2

# 18. Самый убыточный фильм от Paramount Pictures

In [ ]:
answers['18']= '1. K-19: The Widowmaker (tt0267626)'
# "+"

In [ ]:
grouped1 = data[data.production_companies.str.contains(
    "Paramount Pictures", na=False)]
grouped2 = grouped1.sort_values('profit', ascending=True).head(1)
grouped2

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [ ]:
answers["19"] = '5. 2015'
# "+"

In [ ]:
grouped_data = data.groupby(["release_year"])[
    'revenue'].sum().sort_values(ascending=False)
grouped_data.head(5)

# 20. Какой самый прибыльный год для студии Warner Bros?

In [ ]:
answers['20'] = '1. 2014'
# "+"

In [ ]:
grouped1 = data[data.production_companies.str.contains(
    "Warner Bros", na=False)]
grouped_data = grouped1.groupby(["release_year"])[
    'revenue'].sum().sort_values(ascending=False)
grouped_data.head(5)

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [ ]:
answers['21'] = '4. Сентябрь'
# "+"

In [ ]:
data['release_date'] = pd.to_datetime(data['release_date'])
movies_group = data.groupby(data['release_date'].dt.strftime('%B'))[
    'imdb_id'].count().sort_values(ascending=False)
movies_group

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [ ]:
answers["22"] ='2. 450'
# "+"

In [ ]:
data['release_date'] = pd.to_datetime(data['release_date'])
data[data.release_date.dt.month.isin([6, 7, 8, ])]['imdb_id'].count()

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [ ]:
answers['23'] = '5. Peter Jackson'
# "+"

In [ ]:
data_tmp = data[data.release_date.dt.month.isin([1, 2, 12, ])]
grouped2 = data_tmp['director'].str.split(
    '|', expand=True).stack().value_counts()
grouped2

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [ ]:
answers["24"] = '5. Four By Two Productions'
# "+"

In [ ]:

def counter(movie_bd, x):
    '''Создаём функцию, которая вернёт нам количество фильмов с заданными параметрами'''
    data_plot = movie_bd[x].str.cat(sep='|')
    dat = pd.Series(data_plot.split('|'))
    info = dat.value_counts(ascending=False)
    return info


data['lenght_title'] = data.original_title.map(lambda x: len(x))
sum_gen = counter(data, 'production_companies')
for gen in sum_gen.index:
    sum_gen[gen] = data['lenght'][data['production_companies'].map(
        lambda x: True if gen in x else False)].mean()
sum_gen = pd.DataFrame(sum_gen).sort_values(0, ascending=False)
sum_gen

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [ ]:
answers['25'] = '3. Midnight Picture Show'
# "+"

In [ ]:
def counter(movie_bd, x):
    '''Создаём функцию, которая вернёт нам количество фильмов с заданными параметрами'''
    data_plot = movie_bd[x].str.cat(sep='|')
    dat = pd.Series(data_plot.split('|'))
    info = dat.value_counts(ascending=False)
    return info


data['lenght_overview'] = data.overview.map(lambda x: len(x.split(' ')))
sum_gen = counter(data, 'production_companies')
for gen in sum_gen.index:
    sum_gen[gen] = data['lenght_overview'][data['production_companies'].map(
        lambda x: True if gen in x else False)].mean()
sum_gen = pd.DataFrame(sum_gen).sort_values(0, ascending=False)
sum_gen

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [ ]:
answers['26'] = '1. Inside Out, The Dark Knight, 12 Years a Slave'
# "+"

In [ ]:
data[data['vote_average'] > data.quantile(
    0.99, numeric_only=True)['vote_average']]

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


ВАРИАНТ 2

In [ ]:
answers['27'] = '5. Daniel Radcliffe & Rupert Grint'
# "+"

In [ ]:
data[data['vote_average'] > data.quantile(
    0.99, numeric_only=True)['vote_average']]

# Submission

In [ ]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

In [ ]:
# и убедиться что ни чего не пропустил)
len(answers)